# Accelerator Walkthrough: Bringing Your Own Data

This notebook walks you through using `ds_timeseries` as an accelerator for **your** dataset.  
It builds from simple to complex:

| Section | What You'll Do |
|---------|---------------|
| **1. Data Prep** | Load your data, map your fiscal calendar, validate |
| **2. Baseline Models** | Naive, Moving Average, ETS |
| **3. ML Models** | LightGBM, XGBoost, CatBoost |
| **4. Advanced** | Prophet, Direct forecasting, DRFAM |
| **5. Ensembles** | Simple, Weighted, Stacking |
| **6. Hierarchical** | Reconciliation across Parent Customer / Profit Center |
| **7. Scorecard** | Compare everything against your current predictions |

---

### Assumptions

- You have **weekly** sales data at the **Customer-Material** grain  
- You have your company's **fiscal calendar** as a table (dates, week-end dates, fiscal months, etc.)  
- You have **predictions from Nov 25** (FY start) that you want to benchmark against  
- Train/test split: everything **before Nov 25** is train, **Nov 25 onward** is test

---
## 0. Setup

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# ds_timeseries imports
from ds_timeseries.evaluation.metrics import wape, mae, rmse, bias, evaluate_forecast
from ds_timeseries.data.validation import validate_data, classify_demand
from ds_timeseries.features import (
    FiscalCalendarConfig,
    FeatureConfig,
    add_fiscal_features,
    engineer_features,
    rollup_to_fiscal_month,
)

print("Setup complete.")

---
## 1. Data Preparation

### 1a. Load your sales data

The library expects three required columns:

| Column | Type | Description |
|--------|------|-------------|
| `unique_id` | str | One ID per time series (e.g. `"CUST001_MAT123"`) |
| `ds` | datetime | Week-ending date |
| `y` | float | Sales value (quantity or dollars) |

Plus any hierarchy columns you want to keep.

In [ ]:
# === EDIT THIS: Load your data ===
# Examples:
#   raw = pd.read_csv("path/to/weekly_sales.csv")
#   raw = pd.read_parquet("path/to/weekly_sales.parquet")
#   raw = pd.read_excel("path/to/weekly_sales.xlsx")

raw = pd.read_parquet("../data/raw/your_sales_data.parquet")  # <-- replace

raw.head()

### 1b. Map your columns to the expected schema

Rename whatever your columns are called into the standard names.

**Hierarchy mapping for your business:**

```
Total
└── Parent Customer   (your: "parent_customer", "sold_to_group", etc.)
    └── Customer       (your: "sold_to", "ship_to", "customer_number")
        └── Profit Center  (your: "profit_center", "product_line")
            └── Material   (your: "material", "sku", "item_number")
```

The bottom level (`unique_id`) is the **Customer-Material** combination.

In [ ]:
# === EDIT THIS: Map your column names ===

df = raw.rename(columns={
    # Required
    "week_end_date": "ds",          # <-- your date column
    "sales_qty": "y",               # <-- your target column

    # Hierarchy (keep whatever you have)
    "parent_customer": "parent_customer_id",
    "customer_number": "customer_id",
    "profit_center": "profit_center_id",
    "material": "material_id",
})

# Ensure datetime
df["ds"] = pd.to_datetime(df["ds"])

# Build unique_id from Customer + Material
df["unique_id"] = df["customer_id"].astype(str) + "_" + df["material_id"].astype(str)

# Ensure y is numeric
df["y"] = pd.to_numeric(df["y"], errors="coerce").fillna(0)

print(f"Shape: {df.shape}")
print(f"Series: {df['unique_id'].nunique()}")
print(f"Date range: {df['ds'].min()} to {df['ds'].max()}")
print(f"\nColumns: {df.columns.tolist()}")
df.head()

### 1c. Bring your own fiscal calendar

You said you have a table with dates, week-end dates, fiscal month numbers, etc.  
The library needs these columns in the fiscal calendar:

| Column | Type | Required | Notes |
|--------|------|----------|-------|
| `ds` | datetime | **Yes** | Must match your sales data dates exactly |
| `fiscal_year` | int | **Yes** | e.g. 2026 |
| `fiscal_month` | int | **Yes** | 1-12 within the fiscal year |
| `fiscal_quarter` | int | Recommended | 1-4 |
| `fiscal_week` | int | Recommended | 1-52 within the fiscal year |
| `fiscal_week_in_month` | int | Recommended | 1-5 within the fiscal month |
| `fiscal_week_in_quarter` | int | Optional | 1-13 within the quarter |
| `is_fiscal_month_end` | bool | Recommended | Critical for hockey-stick patterns |
| `is_fiscal_quarter_end` | bool | Optional | |
| `is_fiscal_year_end` | bool | Optional | |

Below, load your fiscal calendar and map the column names.

In [ ]:
# === EDIT THIS: Load your fiscal calendar ===

fiscal_raw = pd.read_parquet("../data/raw/your_fiscal_calendar.parquet")  # <-- replace
# fiscal_raw = pd.read_csv("../data/raw/your_fiscal_calendar.csv")
# fiscal_raw = pd.read_excel("../data/raw/your_fiscal_calendar.xlsx")

print("Your fiscal calendar columns:")
print(fiscal_raw.columns.tolist())
fiscal_raw.head(10)

In [ ]:
# === EDIT THIS: Map your fiscal calendar columns ===

fiscal_cal = fiscal_raw.rename(columns={
    "week_end_date": "ds",              # <-- your week-ending date column
    "fiscal_year_number": "fiscal_year",
    "fiscal_month_number": "fiscal_month",
    "fiscal_quarter_number": "fiscal_quarter",
    "fiscal_week_number": "fiscal_week",
    # Add more mappings as needed
})

fiscal_cal["ds"] = pd.to_datetime(fiscal_cal["ds"])

# --- Derive columns you might not have ---

# If you don't have fiscal_week_in_month, derive it:
if "fiscal_week_in_month" not in fiscal_cal.columns:
    fiscal_cal["fiscal_week_in_month"] = (
        fiscal_cal
        .groupby(["fiscal_year", "fiscal_month"])
        .cumcount() + 1
    )

# If you don't have fiscal_week_in_quarter, derive it:
if "fiscal_week_in_quarter" not in fiscal_cal.columns:
    fiscal_cal["fiscal_week_in_quarter"] = (
        fiscal_cal
        .groupby(["fiscal_year", "fiscal_quarter"])
        .cumcount() + 1
    )

# If you don't have is_fiscal_month_end, derive it:
# (last week in each fiscal month = month end)
if "is_fiscal_month_end" not in fiscal_cal.columns:
    max_week_in_month = (
        fiscal_cal
        .groupby(["fiscal_year", "fiscal_month"])["fiscal_week_in_month"]
        .transform("max")
    )
    fiscal_cal["is_fiscal_month_end"] = (
        fiscal_cal["fiscal_week_in_month"] == max_week_in_month
    )

if "is_fiscal_quarter_end" not in fiscal_cal.columns:
    max_week_in_qtr = (
        fiscal_cal
        .groupby(["fiscal_year", "fiscal_quarter"])["fiscal_week_in_quarter"]
        .transform("max")
    )
    fiscal_cal["is_fiscal_quarter_end"] = (
        fiscal_cal["fiscal_week_in_quarter"] == max_week_in_qtr
    )

if "is_fiscal_year_end" not in fiscal_cal.columns:
    max_week_in_year = (
        fiscal_cal
        .groupby("fiscal_year")["fiscal_week"]
        .transform("max")
    )
    fiscal_cal["is_fiscal_year_end"] = (
        fiscal_cal["fiscal_week"] == max_week_in_year
    )

# Keep only the columns the library uses
fiscal_cols = [
    "ds", "fiscal_year", "fiscal_quarter", "fiscal_month", "fiscal_week",
    "fiscal_week_in_month", "fiscal_week_in_quarter",
    "is_fiscal_month_end", "is_fiscal_quarter_end", "is_fiscal_year_end",
]
fiscal_cal = fiscal_cal[[c for c in fiscal_cols if c in fiscal_cal.columns]].drop_duplicates("ds")

print(f"Fiscal calendar: {len(fiscal_cal)} weeks, {fiscal_cal['ds'].min()} to {fiscal_cal['ds'].max()}")
fiscal_cal.head(10)

In [ ]:
# Validate that your fiscal calendar dates align with your sales data dates
sales_dates = set(df["ds"].dt.normalize())
cal_dates = set(fiscal_cal["ds"].dt.normalize())

missing_from_cal = sales_dates - cal_dates
if missing_from_cal:
    print(f"WARNING: {len(missing_from_cal)} sales dates not in your fiscal calendar!")
    print(f"  First few: {sorted(missing_from_cal)[:5]}")
    print(f"  This will cause NaN fiscal features. Fix your calendar or align dates.")
else:
    print("All sales dates found in fiscal calendar.")

### 1d. Validate your data

In [ ]:
validation = validate_data(df[["unique_id", "ds", "y"]])

print(f"Valid: {validation.is_valid}")
if validation.errors:
    print(f"\nErrors:")
    for e in validation.errors:
        print(f"  - {e}")
if validation.warnings:
    print(f"\nWarnings:")
    for w in validation.warnings:
        print(f"  - {w}")
print(f"\nStats:")
for k, v in validation.stats.items():
    print(f"  {k}: {v}")

In [ ]:
# Classify demand patterns - tells you which series are intermittent/lumpy
demand_classes = classify_demand(df[["unique_id", "ds", "y"]])

print("Demand pattern distribution:")
print(demand_classes["category"].value_counts())
print()
print(f"Avg zero %: {demand_classes['zero_pct'].mean():.1%}")

### 1e. Define train/test split at fiscal year start (Nov 25)

This is the critical split. Everything before your FY start is training data.  
Everything from Nov 25 onward is the test period you'll compare models against your existing predictions.

In [ ]:
# === EDIT THIS: Your fiscal year cutoff date ===
CUTOFF_DATE = pd.Timestamp("2025-11-25")

# If your week-end dates don't land exactly on Nov 25, find the nearest one:
all_dates = sorted(df["ds"].unique())
cutoff_idx = np.searchsorted(all_dates, CUTOFF_DATE)
actual_cutoff = all_dates[cutoff_idx]  # first test date
print(f"Requested cutoff: {CUTOFF_DATE.date()}")
print(f"Actual first test date: {pd.Timestamp(actual_cutoff).date()}")

train = df[df["ds"] < actual_cutoff].copy()
test = df[df["ds"] >= actual_cutoff].copy()

# How many weeks in the test period?
test_weeks = test["ds"].nunique()
HORIZON = test_weeks

print(f"\nTrain: {train['ds'].nunique()} weeks, {train['ds'].min().date()} to {train['ds'].max().date()}")
print(f"Test:  {test_weeks} weeks, {test['ds'].min().date()} to {test['ds'].max().date()}")
print(f"Series: {train['unique_id'].nunique()} in train, {test['unique_id'].nunique()} in test")
print(f"\nHORIZON = {HORIZON} weeks")

### 1f. Load your existing predictions (the benchmark to beat)

In [ ]:
# === EDIT THIS: Load your existing predictions ===
# These are the forecasts your team made starting Nov 25

existing_preds_raw = pd.read_parquet("../data/raw/your_predictions.parquet")  # <-- replace

# Map to standard format
existing_preds = existing_preds_raw.rename(columns={
    "week_end_date": "ds",
    "predicted_qty": "yhat",
    # map customer/material to build unique_id
    "customer_number": "customer_id",
    "material": "material_id",
})
existing_preds["ds"] = pd.to_datetime(existing_preds["ds"])
existing_preds["unique_id"] = (
    existing_preds["customer_id"].astype(str) + "_" + existing_preds["material_id"].astype(str)
)

# Score your existing predictions against actuals
benchmark = test.merge(existing_preds[["unique_id", "ds", "yhat"]], on=["unique_id", "ds"], how="inner")
benchmark_wape = wape(benchmark["y"], benchmark["yhat"])
benchmark_mae = mae(benchmark["y"], benchmark["yhat"])

print(f"Your existing predictions (benchmark to beat):")
print(f"  WAPE: {benchmark_wape:.2%}")
print(f"  MAE:  {benchmark_mae:.2f}")
print(f"  Matched rows: {len(benchmark)} of {len(test)} test rows")

---
## 2. Baseline Models (Start Simple)

Always start with baselines. They're fast, require no tuning, and set the floor.  
If ML models can't beat these, the data doesn't have learnable patterns.

In [ ]:
from ds_timeseries.models import (
    NaiveForecaster,
    MovingAverageForecaster,
    SeasonalNaiveForecaster,
    ETSForecaster,
)

# We'll collect all results here
results = {}

def score_model(name, model, train_df, test_df, horizon):
    """Fit model, predict, score against test, store results."""
    model.fit(train_df)
    preds = model.predict(horizon=horizon)

    merged = test_df.merge(preds, on=["unique_id", "ds"], how="inner")
    if len(merged) == 0:
        print(f"  {name}: No matching predictions! Check date alignment.")
        return None

    scores = evaluate_forecast(merged["y"], merged["yhat"])
    scores["name"] = name
    scores["matched_rows"] = len(merged)
    results[name] = scores

    print(f"  {name}: WAPE={scores['wape']:.2%}, MAE={scores['mae']:.2f}, Bias={scores['bias']:.2f}")
    return preds

In [ ]:
# --- Baselines (fast, no features needed) ---

train_base = train[["unique_id", "ds", "y"]].copy()
test_base = test[["unique_id", "ds", "y"]].copy()

print("Running baseline models...\n")

# 1. Naive (last value repeated)
score_model("Naive", NaiveForecaster(), train_base, test_base, HORIZON)

# 2. Moving Average (4-week window)
score_model("MA(4)", MovingAverageForecaster(window=4), train_base, test_base, HORIZON)

# 3. Moving Average (8-week window)
score_model("MA(8)", MovingAverageForecaster(window=8), train_base, test_base, HORIZON)

# 4. Moving Average (13-week window = 1 quarter)
score_model("MA(13)", MovingAverageForecaster(window=13), train_base, test_base, HORIZON)

# 5. Seasonal Naive (same week last year) - needs 52+ weeks of history
score_model("SeasonalNaive(52)", SeasonalNaiveForecaster(season_length=52), train_base, test_base, HORIZON)

# 6. ETS (Exponential Smoothing - auto-selects best spec)
score_model("ETS", ETSForecaster(season_length=52), train_base, test_base, HORIZON)

### Quick check: How do baselines compare to your existing predictions?

In [ ]:
print(f"{'Model':<25} {'WAPE':>8} {'MAE':>8} {'vs Benchmark':>14}")
print("-" * 57)
print(f"{'Your Predictions':<25} {benchmark_wape:>8.2%} {benchmark_mae:>8.2f} {'baseline':>14}")
print("-" * 57)
for name, scores in sorted(results.items(), key=lambda x: x[1]["wape"]):
    delta = scores["wape"] - benchmark_wape
    arrow = "+" if delta > 0 else ""
    print(f"{name:<25} {scores['wape']:>8.2%} {scores['mae']:>8.2f} {arrow}{delta:>13.2%}")

---
## 3. ML Models (LightGBM, XGBoost, CatBoost)

These need feature engineering. The library handles this for you, including your fiscal calendar features.

### 3a. Configure features with your fiscal calendar

In [ ]:
from ds_timeseries.models import LightGBMForecaster, XGBoostForecaster, CatBoostForecaster

# Feature config - the fiscal_config here is only used if you DON'T pass a
# pre-built fiscal_calendar. Since we have our own calendar, we'll pass it
# directly to engineer_features and the ML models will pick it up.
#
# But we still set a config so the FeatureConfig dataclass is happy:
feature_config = FeatureConfig(
    lags=[1, 2, 4, 8, 13, 26, 52],
    rolling_windows=[4, 13, 26, 52],
    rolling_aggs=["mean", "std"],
    diff_periods=[1, 4, 52],
    pct_change_periods=[1, 52],
    include_expanding=True,
    fiscal_config=FiscalCalendarConfig(),  # placeholder, we override with our calendar
    include_calendar_features=True,
)

print("Feature config ready.")
print(f"  Lags: {feature_config.lags}")
print(f"  Rolling windows: {feature_config.rolling_windows}")
print(f"  Total features: ~33 numeric + fiscal + calendar")

### 3b. Preview the engineered features

Run this on a small sample first to verify everything looks right.

In [ ]:
# Preview features on a small sample
sample_ids = train["unique_id"].unique()[:3]
sample = train[train["unique_id"].isin(sample_ids)][["unique_id", "ds", "y"]].copy()

sample_feat = engineer_features(
    sample,
    config=feature_config,
    fiscal_calendar=fiscal_cal,  # <-- YOUR fiscal calendar
)

print(f"Columns ({len(sample_feat.columns)}):")
print(sample_feat.columns.tolist())
print(f"\nShape: {sample_feat.shape}")
print(f"NaN rows (from lag warmup): {sample_feat.isna().any(axis=1).sum()}")

# Check fiscal features look right
fiscal_preview = sample_feat[["ds", "fiscal_year", "fiscal_quarter", "fiscal_month",
                               "fiscal_week", "is_fiscal_month_end"]].drop_duplicates("ds").tail(10)
print("\nFiscal feature preview (last 10 weeks):")
fiscal_preview

### 3c. Train ML models

Note: The ML forecasters call `engineer_features` internally. We pass our fiscal
calendar through the model's feature_config. However, since the models generate
the calendar internally from the config, and we want to use **our own** calendar,
we need to merge fiscal features onto the data **before** passing it to the models,
then tell the model not to regenerate them.

**Strategy**: Pre-merge fiscal features from your calendar, then let the ML
models add the lag/rolling features on top.

In [ ]:
# Pre-merge fiscal features from YOUR calendar onto train and test
train_with_fiscal = add_fiscal_features(train[["unique_id", "ds", "y"]], fiscal_calendar=fiscal_cal)
test_with_fiscal = add_fiscal_features(test[["unique_id", "ds", "y"]], fiscal_calendar=fiscal_cal)

print(f"Train with fiscal: {train_with_fiscal.shape}")
print(f"Test with fiscal: {test_with_fiscal.shape}")
print(f"\nFiscal columns added: {[c for c in train_with_fiscal.columns if 'fiscal' in c.lower()]}")

In [ ]:
# --- LightGBM (Tweedie) ---
print("Training LightGBM (Tweedie)...")
lgb_model = LightGBMForecaster(
    feature_config=feature_config,
    use_tweedie=True,
    strategy="recursive",
)
score_model("LightGBM", lgb_model, train_with_fiscal, test_with_fiscal, HORIZON)

In [ ]:
# --- XGBoost ---
print("Training XGBoost...")
xgb_model = XGBoostForecaster(
    feature_config=feature_config,
    strategy="recursive",
)
score_model("XGBoost", xgb_model, train_with_fiscal, test_with_fiscal, HORIZON)

In [ ]:
# --- CatBoost ---
print("Training CatBoost...")
cat_model = CatBoostForecaster(
    feature_config=feature_config,
)
score_model("CatBoost", cat_model, train_with_fiscal, test_with_fiscal, HORIZON)

### 3d. Direct forecasting variant

Direct strategy trains a **separate model for each horizon step** (week 1, week 2, ...).  
Avoids error accumulation from recursive predictions. The M5 1st place used both.

In [ ]:
# --- LightGBM Direct ---
print(f"Training LightGBM Direct (one model per week, {HORIZON} models)...")
lgb_direct = LightGBMForecaster(
    feature_config=feature_config,
    use_tweedie=True,
    strategy="direct",
)
lgb_direct.fit(train_with_fiscal, horizon=HORIZON)
preds_direct = lgb_direct.predict(horizon=HORIZON)

merged_direct = test_with_fiscal.merge(preds_direct, on=["unique_id", "ds"], how="inner")
scores_direct = evaluate_forecast(merged_direct["y"], merged_direct["yhat"])
scores_direct["name"] = "LightGBM (Direct)"
scores_direct["matched_rows"] = len(merged_direct)
results["LightGBM (Direct)"] = scores_direct
print(f"  LightGBM (Direct): WAPE={scores_direct['wape']:.2%}, MAE={scores_direct['mae']:.2f}")

---
## 4. Advanced Models

### 4a. Prophet

In [ ]:
from ds_timeseries.models import ProphetForecaster

print("Training Prophet...")
prophet_model = ProphetForecaster(
    yearly_seasonality=True,
    weekly_seasonality=False,
)
score_model("Prophet", prophet_model, train_base, test_base, HORIZON)

### 4b. Intermittent demand models

If your demand classification showed a lot of intermittent/lumpy series,  
these specialized models may outperform on those series.

In [ ]:
from ds_timeseries.models import CrostonForecaster, SBAForecaster, TSBForecaster

print("Training intermittent demand models...\n")

score_model("Croston", CrostonForecaster(), train_base, test_base, HORIZON)
score_model("SBA", SBAForecaster(), train_base, test_base, HORIZON)
score_model("TSB", TSBForecaster(), train_base, test_base, HORIZON)

### 4c. DRFAM Ensemble (M5 1st Place approach)

Averages Direct and Recursive forecasts from the same model class.  
This is the strategy used by the M5 competition winner.

In [ ]:
from ds_timeseries.models import DRFAMEnsemble

print(f"Training DRFAM (Direct + Recursive averaging)...")
drfam = DRFAMEnsemble(
    model_class=LightGBMForecaster,
    model_params={"feature_config": feature_config, "use_tweedie": True},
    use_direct=True,
    use_recursive=True,
)
drfam.fit(train_with_fiscal, horizon=HORIZON)
preds_drfam = drfam.predict(horizon=HORIZON)

merged_drfam = test_with_fiscal.merge(preds_drfam, on=["unique_id", "ds"], how="inner")
scores_drfam = evaluate_forecast(merged_drfam["y"], merged_drfam["yhat"])
scores_drfam["name"] = "DRFAM (LightGBM)"
scores_drfam["matched_rows"] = len(merged_drfam)
results["DRFAM (LightGBM)"] = scores_drfam
print(f"  DRFAM: WAPE={scores_drfam['wape']:.2%}, MAE={scores_drfam['mae']:.2f}")

---
## 5. Ensemble Models

Combine multiple models. Often the best approach in practice.

In [ ]:
from ds_timeseries.models import SimpleEnsemble, WeightedEnsemble, StackingEnsemble

# Base models for ensembles
ensemble_models = [
    LightGBMForecaster(feature_config=feature_config, use_tweedie=True),
    XGBoostForecaster(feature_config=feature_config),
    CatBoostForecaster(feature_config=feature_config),
]

In [ ]:
# --- Simple Ensemble (equal average) ---
print("Training Simple Ensemble (equal weights)...")
simple_ens = SimpleEnsemble(models=ensemble_models)
score_model("SimpleEnsemble", simple_ens, train_with_fiscal, test_with_fiscal, HORIZON)

In [ ]:
# --- Weighted Ensemble (learns weights from CV) ---
print("Training Weighted Ensemble (CV-optimized weights)...")
weighted_ens = WeightedEnsemble(
    models=[
        LightGBMForecaster(feature_config=feature_config, use_tweedie=True),
        XGBoostForecaster(feature_config=feature_config),
        CatBoostForecaster(feature_config=feature_config),
    ],
    cv_folds=3,
    metric="wape",
)
score_model("WeightedEnsemble", weighted_ens, train_with_fiscal, test_with_fiscal, HORIZON)

In [ ]:
# --- Stacking Ensemble (meta-learner) ---
print("Training Stacking Ensemble (Ridge meta-learner)...")
stacking_ens = StackingEnsemble(
    models=[
        LightGBMForecaster(feature_config=feature_config, use_tweedie=True),
        XGBoostForecaster(feature_config=feature_config),
        CatBoostForecaster(feature_config=feature_config),
    ],
    cv_folds=3,
)
score_model("StackingEnsemble", stacking_ens, train_with_fiscal, test_with_fiscal, HORIZON)

---
## 6. Hierarchical Reconciliation

Your hierarchy:
```
Total
└── Parent Customer
    └── Customer
        └── Profit Center
            └── Material
                └── unique_id (Customer_Material)
```

Reconciliation ensures forecasts **sum correctly up the hierarchy**.  
Bottom-up is simplest: forecast at the granular level, aggregate up.  
MinTrace is optimal: adjusts all levels to minimize total error.

**Note**: This section requires that your data has the hierarchy columns  
(`parent_customer_id`, `customer_id`, `profit_center_id`, `material_id`).

In [ ]:
from ds_timeseries.reconciliation.hierarchy import HierarchySpec, create_hierarchy_from_data
from ds_timeseries.reconciliation.methods import reconcile_forecasts

# === EDIT THIS: Define your hierarchy (bottom to top) ===
# List columns from most granular to most aggregated

hierarchy_cols = [
    "material_id",          # Most granular (within a customer)
    "profit_center_id",     # Profit Center groups materials
    "customer_id",          # Customer groups profit centers
    "parent_customer_id",   # Parent Customer groups customers
]

# Check which columns actually exist in your data
available = [c for c in hierarchy_cols if c in train.columns]
missing = [c for c in hierarchy_cols if c not in train.columns]

if missing:
    print(f"Missing hierarchy columns: {missing}")
    print(f"Available: {available}")
    print(f"Skipping reconciliation - add these columns to your data or adjust the list above.")
else:
    hierarchy = HierarchySpec(levels=available, bottom_level="unique_id")
    print(f"Hierarchy defined: {' > '.join(reversed(available))} > unique_id")
    print(f"Bottom-level series: {train['unique_id'].nunique()}")

In [ ]:
# Get bottom-level forecasts from best model so far
# (Re-use whichever performed best above)

best_model_name = min(results, key=lambda k: results[k]["wape"])
print(f"Using best model for reconciliation: {best_model_name}")

# Re-fit the best model and get predictions
# (We need the raw predictions DataFrame, not just scores)
best_model = LightGBMForecaster(feature_config=feature_config, use_tweedie=True)
best_model.fit(train_with_fiscal)
bottom_preds = best_model.predict(horizon=HORIZON)

print(f"Bottom-level predictions: {len(bottom_preds)} rows")

In [ ]:
# Build aggregation matrix and reconcile
if not missing:  # only if hierarchy columns exist
    # Need hierarchy columns in the data for building the matrix
    train_hier = train[["unique_id", "ds", "y"] + available].copy()
    test_hier = test[["unique_id", "ds", "y"] + available].copy()

    hierarchy.build_aggregation_matrix(train_hier)

    # Bottom-Up reconciliation
    reconciled_bu = reconcile_forecasts(
        bottom_preds, test_hier, hierarchy, method="bottom_up"
    )
    print(f"Bottom-Up reconciled: {len(reconciled_bu)} rows across all levels")

    # Score at bottom level (same as unreconciled for bottom-up)
    recon_bottom = reconciled_bu[reconciled_bu["level"] == "unique_id"]
    recon_merged = test_hier.merge(
        recon_bottom[["unique_id", "ds", "yhat"]], on=["unique_id", "ds"], how="inner"
    )
    print(f"  Bottom-Up WAPE: {wape(recon_merged['y'], recon_merged['yhat']):.2%}")

    # MinTrace reconciliation (optimal)
    try:
        reconciled_mt = reconcile_forecasts(
            bottom_preds, test_hier, hierarchy, method="ols"
        )
        mt_bottom = reconciled_mt[reconciled_mt["level"] == "unique_id"]
        mt_merged = test_hier.merge(
            mt_bottom[["unique_id", "ds", "yhat"]], on=["unique_id", "ds"], how="inner"
        )
        print(f"  MinTrace (OLS) WAPE: {wape(mt_merged['y'], mt_merged['yhat']):.2%}")
    except Exception as e:
        print(f"  MinTrace failed (may need more data): {e}")

---
## 7. Final Scorecard: All Models vs Your Predictions

The moment of truth. Every model scored against the same test period,  
compared to your existing predictions from November 25.

In [ ]:
# Build the scorecard
scorecard = pd.DataFrame(results).T

# Add benchmark row
benchmark_row = pd.DataFrame([{
    "name": "YOUR PREDICTIONS",
    "wape": benchmark_wape,
    "mae": benchmark_mae,
    "rmse": np.nan,
    "bias": np.nan,
    "matched_rows": len(benchmark),
}])
scorecard = pd.concat([benchmark_row, scorecard], ignore_index=True)

# Sort by WAPE
scorecard = scorecard.sort_values("wape").reset_index(drop=True)
scorecard["rank"] = range(1, len(scorecard) + 1)

# Add comparison to benchmark
scorecard["wape_vs_benchmark"] = scorecard["wape"] - benchmark_wape
scorecard["wape_improvement_%"] = -(scorecard["wape_vs_benchmark"] / benchmark_wape * 100)

display_cols = ["rank", "name", "wape", "mae", "bias", "wape_improvement_%"]
display_cols = [c for c in display_cols if c in scorecard.columns]

print("=" * 80)
print("FINAL SCORECARD")
print(f"Test period: {test['ds'].min().date()} to {test['ds'].max().date()} ({HORIZON} weeks)")
print("=" * 80)
print()
scorecard[display_cols]

In [ ]:
# Visual comparison
try:
    import plotly.express as px

    fig = px.bar(
        scorecard.sort_values("wape"),
        x="wape",
        y="name",
        orientation="h",
        title=f"Model Comparison: WAPE (lower is better) - {HORIZON}-week test",
        labels={"wape": "WAPE", "name": ""},
        color="wape",
        color_continuous_scale="RdYlGn_r",
    )
    # Add vertical line for benchmark
    fig.add_vline(x=benchmark_wape, line_dash="dash", line_color="red",
                  annotation_text="Your Predictions")
    fig.update_layout(height=max(400, len(scorecard) * 35), showlegend=False)
    fig.show()
except ImportError:
    print("Install plotly for interactive charts: pip install plotly")

---
## 8. Roll Up to Fiscal Months

Leadership often wants to see accuracy at the **fiscal month** level, not weekly.  
Use `rollup_to_fiscal_month` with **your own** fiscal calendar.

In [ ]:
# Get predictions from the best model
best_name = scorecard.loc[scorecard["name"] != "YOUR PREDICTIONS"].iloc[0]["name"]
print(f"Best model: {best_name}")

# Re-generate predictions for rollup
best_model = LightGBMForecaster(feature_config=feature_config, use_tweedie=True)
best_model.fit(train_with_fiscal)
best_preds = best_model.predict(horizon=HORIZON)

# Merge actuals + predictions for rollup
test_merged = test[["unique_id", "ds", "y"]].merge(
    best_preds[["unique_id", "ds", "yhat"]], on=["unique_id", "ds"], how="inner"
)

# Roll up to fiscal months using YOUR calendar
monthly = rollup_to_fiscal_month(
    test_merged,
    value_cols=["y", "yhat"],
    fiscal_calendar=fiscal_cal,  # <-- YOUR fiscal calendar
)

# Monthly WAPE
complete = monthly[monthly["is_complete"]]
print(f"\nMonthly WAPE (complete months only):")
for _, row in complete.groupby(["fiscal_year", "fiscal_month"]).agg(
    {"y": "sum", "yhat": "sum"}
).iterrows():
    m_wape = abs(row["y"] - row["yhat"]) / abs(row["y"]) if row["y"] != 0 else float("inf")
    print(f"  FY{_[0]} M{_[1]:02d}: WAPE={m_wape:.2%}  (actual={row['y']:,.0f}, forecast={row['yhat']:,.0f})")

overall_monthly_wape = wape(complete["y"], complete["yhat"])
print(f"\n  Overall monthly WAPE: {overall_monthly_wape:.2%}")

---
## 9. Neural Models (Optional)

Requires `neuralforecast` to be installed: `pip install neuralforecast`  
These are heavier, slower, and may not beat gradient boosting on tabular data.  
But worth trying, especially TFT for long horizons.

In [ ]:
try:
    from ds_timeseries.models.neural import (
        NBEATSForecaster,
        NHITSForecaster,
        NeuralConfig,
    )

    neural_config = NeuralConfig(
        input_size=52,       # look back 52 weeks
        max_steps=500,       # training steps
        learning_rate=1e-3,
        batch_size=32,
        accelerator="auto",  # "gpu" if you have one, else "cpu"
    )

    print("Training N-BEATS...")
    nbeats = NBEATSForecaster(horizon=HORIZON, config=neural_config)
    score_model("N-BEATS", nbeats, train_base, test_base, HORIZON)

    print("Training N-HITS...")
    nhits = NHITSForecaster(horizon=HORIZON, config=neural_config)
    score_model("N-HITS", nhits, train_base, test_base, HORIZON)

except ImportError:
    print("neuralforecast not installed. Skipping neural models.")
    print("Install with: pip install neuralforecast")

---
## 10. Hyperparameter Tuning (Optional)

Once you've identified the top 1-2 model types, tune them for your data.

In [ ]:
from ds_timeseries.models.tuning import tune_lightgbm, tune_xgboost

# Only tune on training data (uses internal time-series CV, no leakage)
print("Tuning LightGBM (this may take a while)...")
tune_result = tune_lightgbm(
    train_with_fiscal,
    feature_config=feature_config,
    n_iter=20,    # number of random configurations to try
    n_folds=3,
)

print(f"\nBest params: {tune_result.best_params}")
print(f"Best CV WAPE: {tune_result.best_score:.2%}")

# Score tuned model on the real test set
if tune_result.best_model:
    score_model("LightGBM (Tuned)", tune_result.best_model, train_with_fiscal, test_with_fiscal, HORIZON)

---
## Quick Reference: Adapting to Your Data

### Checklist

| Step | What to Change | Where |
|------|---------------|-------|
| Load data | File path and format | Section 1a |
| Column mapping | Rename your columns to `unique_id`, `ds`, `y` | Section 1b |
| Fiscal calendar | Load your table, map columns, derive missing flags | Section 1c |
| Cutoff date | Set `CUTOFF_DATE` to your FY start | Section 1e |
| Benchmark | Load your existing predictions | Section 1f |
| Hierarchy columns | Edit `hierarchy_cols` list | Section 6 |

### Hierarchy Notes

**Your business hierarchy:**
```
Total
└── Parent Customer   →  parent_customer_id
    └── Customer       →  customer_id
        └── Profit Center  →  profit_center_id
            └── Material   →  material_id
                └── Customer_Material  →  unique_id
```

**Key points:**
- `unique_id` = `customer_id` + `_` + `material_id` (the bottom level)
- The hierarchy defines how forecasts **aggregate up**
- Material rolls up to Profit Center (product dimension)
- Customer rolls up to Parent Customer (customer dimension)
- This is a **cross-hierarchy** (product x customer), which is more complex than a simple tree
- For cross-hierarchies, you may want to reconcile each dimension separately,
  or flatten to a single tree by combining: `Parent Customer > Customer > Profit Center > Material`

### Fiscal Calendar Notes

**What you bring:**
- Your fiscal calendar table with dates, week-end dates, fiscal months, etc.
- The `ds` column in your calendar MUST match the `ds` dates in your sales data exactly

**What the library derives if missing:**
- `fiscal_week_in_month`: cumulative count of weeks per fiscal month
- `fiscal_week_in_quarter`: cumulative count of weeks per fiscal quarter
- `is_fiscal_month_end`: last week in each fiscal month (drives hockey-stick detection)
- `is_fiscal_quarter_end`, `is_fiscal_year_end`: period boundary flags

**Why `is_fiscal_month_end` matters:**  
Sales surge at fiscal period ends (the "hockey stick"). This boolean flag lets ML models
learn that pattern directly. It's typically the single most important fiscal feature.

### Train/Test Split Notes

- The split is at your FY start (Nov 25) - all models see the same history
- No cross-validation on the test period - that's your holdout for honest comparison
- Cross-validation happens **within training data only** (for tuning & weighted ensembles)
- This mirrors a real production scenario: train on history, forecast the fiscal year

In [ ]:
# Save the scorecard for reference
scorecard.to_csv("../data/raw/model_scorecard.csv", index=False)
print("Scorecard saved to data/raw/model_scorecard.csv")